In [43]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from typing import TypedDict
from dotenv import load_dotenv
import os

In [44]:
load_dotenv()

True

In [45]:
api_key=os.environ.get("GROQ_API_KEY")

In [46]:
llm = ChatGroq(
    model="openai/gpt-oss-20b",
    api_key=api_key,
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    # other params...
)

In [47]:
class BlogState(TypedDict):
    title:str
    outline:str
    content:str

In [48]:
# Create outline and blog function
def create_outline(state:BlogState) -> BlogState:
    # fetch file
    title=state['title']

    # call llm gen outline
    prompt=f"Generate outline for a blog on the topic -{title}"

    outline=llm.invoke(prompt).content

    state['outline'] = outline

    return state

In [49]:
def create_blog(state:BlogState) -> BlogState:
    title=state['title']
    outline=state['outline']
    prompt=f"write a detailed blog on title using the following outline {outline}"
    content=llm.invoke(prompt).content
    state['content']=content
    return state


In [50]:
# execute 
graph=StateGraph(BlogState)

# nodes
graph.add_node('create_outline',create_outline)
graph.add_node('create_blog',create_blog)

# edges
graph.add_edge(START,'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog',END)

workflow=graph.compile()

In [51]:
initial_state={'title':'Rise of AI in India'}
final_state=workflow.invoke(initial_state)
print(final_state)

{'title': 'Rise of AI in India', 'outline': '## Blog Outline  \n**Title:** *Rise of AI in India: From Startup Hubs to Smart Nation*\n\n---\n\n### 1. Introduction  \n- **Hook:** A striking statistic or anecdote (e.g., “India’s AI market is projected to hit $10\u202fB by 2025”).  \n- **Purpose:** Explain why the AI boom matters for India’s economy, society, and global standing.  \n- **Roadmap:** Briefly outline the sections that will follow.\n\n---\n\n### 2. Historical Context  \n- **Early AI Efforts (1990s‑2000s):**  \n  - Academic research (e.g., IITs, NITs).  \n  - First AI labs and government projects.  \n- **The Digital Revolution (2010‑2015):**  \n  - Mobile penetration, cloud adoption.  \n  - Rise of data as a commodity.  \n- **Catalysts for the Current Boom:**  \n  - Government’s “Digital India” and “Startup India” initiatives.  \n  - Global AI trends and the “AI race” narrative.\n\n---\n\n### 3. Current Landscape of AI in India  \n- **Key Players:**  \n  - **Large Corporations:*

In [52]:
print(final_state['outline'])

## Blog Outline  
**Title:** *Rise of AI in India: From Startup Hubs to Smart Nation*

---

### 1. Introduction  
- **Hook:** A striking statistic or anecdote (e.g., “India’s AI market is projected to hit $10 B by 2025”).  
- **Purpose:** Explain why the AI boom matters for India’s economy, society, and global standing.  
- **Roadmap:** Briefly outline the sections that will follow.

---

### 2. Historical Context  
- **Early AI Efforts (1990s‑2000s):**  
  - Academic research (e.g., IITs, NITs).  
  - First AI labs and government projects.  
- **The Digital Revolution (2010‑2015):**  
  - Mobile penetration, cloud adoption.  
  - Rise of data as a commodity.  
- **Catalysts for the Current Boom:**  
  - Government’s “Digital India” and “Startup India” initiatives.  
  - Global AI trends and the “AI race” narrative.

---

### 3. Current Landscape of AI in India  
- **Key Players:**  
  - **Large Corporations:** Infosys, Wipro, TCS, HCL, IBM India.  
  - **Startups & Scale‑ups:** Haptik